# 0. Import Packages

In [10]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
import time
import re
import pandas as pd
from datetime import datetime
from datetime import date
from datetime import date, timedelta
import random
import threading

# 1. Functions

## 1.1. Loop Through Dates

In [11]:
def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days) + 1):
        yield start_date + timedelta(n)

# 2. Scraping News Media

## 2.1. Wall Street Journal

### 2.1.1. Scraping Function

In [35]:
def getWallStreetJournal(username, password, start_date, end_date):
    #put dates in date format
    start_date = datetime.strptime(start_date, "%d/%m/%Y")
    end_date = datetime.strptime(end_date, "%d/%m/%Y")
    
    #Define list of variables to store 
    WSJ_article_content = []
    WSJ_date = []
    WSJ_link = []
    WSJ_title = []
    
    #Go to the website
    fireFoxOptions = webdriver.FirefoxOptions()
    fireFoxOptions.add_argument('--ignore-certificate-errors')
    fireFoxOptions.add_argument('--allow-running-insecure-content')
    fireFoxOptions.add_argument("--headless")
    fireFoxOptions.add_argument("--disable-gpu")
    fireFoxOptions.add_argument("--no-sandbox")
    #fireFoxOptions.add_argument("-width=1800");
    #fireFoxOptions.add_argument("-height=6000");
    user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36'
    fireFoxOptions.add_argument(f'user-agent={user_agent}')
    driver_WSJ = webdriver.Firefox(options=fireFoxOptions)
    #driver_WSJ = webdriver.Firefox()
    driver_WSJ.get("https://sso.accounts.dowjones.com/login-page?op=localop&scope=openid%20idp_id%20roles%20email%20given_name%20family_name%20djid%20djUsername%20djStatus%20trackid%20tags%20prts%20suuid%20createTimestamp&client_id=5hssEAdMy0mJTICnJNvC9TXEw3Va7jfO&response_type=code&redirect_uri=https%3A%2F%2Faccounts.wsj.com%2Fauth%2Fsso%2Flogin&nonce=ab6a473a-cfa6-4714-8fad-b6dff98f5f18&ui_locales=en-us-x-wsj-223-2&mars=-1&ns=prod%2Faccounts-wsj&state=8rChOTDzC_Y_AK-i.TJAixN_XjsWxwUEEPoHg2OPCaX6qRBu4nGSk5fqLliY4H0B5F7gj_57-XH-YBWGS&protocol=oauth2&client=5hssEAdMy0mJTICnJNvC9TXEw3Va7jfO#!/signin")
    
    driver_WSJ.get_screenshot_as_file("login2.png")
    
    #put in user_name:
    time.sleep(3) #makes sure field is fully loaded
    username_field = driver_WSJ.find_element(By.XPATH, "/html/body/div[1]/div[2]/div/div/div/div[1]/div[1]/form/div[2]/div[1]/div[2]/input")
    username_field.send_keys(username)
    
    #continue to password:
    driver_WSJ.find_element(By.XPATH, "/html/body/div[1]/div[2]/div/div/div/div[1]/div[1]/form/div[2]/div[6]/div[1]/button[2]").click()

    #input password:
    time.sleep(3) #makes sure field is fully loaded
    password_field = driver_WSJ.find_element(By.XPATH, "//*[@id='password-login-password']")
    password_field.send_keys(password)

    #click on sign in:
    driver_WSJ.find_element(By.XPATH, "//*[@id='password-login']/div/form/div/div[5]/div[1]/button").click()
    
    try:
        #accept cookies
        time.sleep(10)
        driver_WSJ.switch_to.frame(WebDriverWait(driver_WSJ,30).until(EC.presence_of_element_located((By.ID, 'sp_message_iframe_718122'))))
        driver_WSJ.find_element(By.CSS_SELECTOR, "button.message-component:nth-child(2)").click()
        driver_WSJ.switch_to.default_content() 
        time.sleep(3)
    except:
        pass

    scrape_time = time.time() #starting the timing
    
    for single_date in daterange(start_date, end_date):
        driver_WSJ.get_screenshot_as_file("articles_page2.png")
        date = single_date.strftime("%d/%m/%Y")
        print(date)
        year = "%02d" % (int(date.split('/')[2]),)
        month = "%02d" % (int(date.split('/')[1]),)
        day = "%02d" % (int(date.split('/')[0]),)
        page = "%02d" % (1,) #we assume there is always at least one page
        url = f"https://www.wsj.com/news/archive/{year}/{month}/{day}?page={page}"
        driver_WSJ.get(url) #go to the page in the WSJ archive for the given date
        
        #collect the article urls for the articles on the first page
        articles = WebDriverWait(driver_WSJ,10).until(EC.presence_of_all_elements_located((By.TAG_NAME, "article")))
        for a in range(0,len(articles)):
            WSJ_link.append(articles[a].find_element(By.TAG_NAME, "a").get_attribute('href'))
            WSJ_date.append(date)

        #check if there are multiple pages and if so, visit these as well
        pages = WebDriverWait(driver_WSJ, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "WSJTheme--pagepicker-total--Kl350I1l ")))
        page_total = re.findall(r'\d+', pages.text)
        nr_pages = int(page_total[0])

        for p in range(1,nr_pages):
            page = "%02d" % (p,)
            url = f"https://www.wsj.com/news/archive/{year}/{month}/{day}?page={page}"
            trys = 0
            while(trys < 100):
                try:
                    driver_WSJ.get(url) #go to the page in the WSJ archive for the given date
                    break
                except:
                    trys =+ 1

            #get the article urls for the articles on the other pages, if there are more than 1
            articles = WebDriverWait(driver_WSJ,10).until(EC.presence_of_all_elements_located((By.TAG_NAME, "article")))
            for a in range(0,len(articles)):
                WSJ_link.append(articles[a].find_element(By.TAG_NAME, "a").get_attribute('href'))
                WSJ_date.append(date)

    for a in range(0,len(WSJ_link)):
        trys = 0
        while(trys < 100):
            try:
                #go to every article
                driver_WSJ.get(WSJ_link[a])
                break
            except:
                trys += 1

        try:
            #extract the content and add to a variables
            time.sleep(10)
            driver_WSJ.get_screenshot_as_file("article2.png")
            content = WebDriverWait(driver_WSJ, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'article-content  ')))
            text = content.find_elements(By.TAG_NAME, 'p')
            article_text = ""
            for t in range(0,len(text)):
                article_text += " " + text[t].text
            
            WSJ_article_content.append(article_text) #add text to the list
            #WSJ_article_content.append(content.text) #add text to the list

        except: #we don't have access to the article
            WSJ_article_content.append("NO ACCESS")
            pass #go back to page with all articles

        try:
            #collect the title as well
            title = WebDriverWait(driver_WSJ,10).until(EC.presence_of_element_located((By.CLASS_NAME, "wsj-article-headline"))).text
            WSJ_title.append(title)
        except:
            try:
                #there are 2 main formats in which the titles are present in the HTML
                title = WebDriverWait(driver_WSJ,10).until(EC.presence_of_element_located((By.CLASS_NAME, "bigTop__hed"))).text
                WSJ_title.append(title)

            except:
                #add error if there is some unexpected layout -> this way our arrays will be of the same length and we will be able
                #to construct a dataframe in the end
                WSJ_title.append("ERROR")
                pass

    print("--- %s seconds ---" % (time.time() - scrape_time)) #how long did we scrape? 

    driver_WSJ.quit()
    #Make the dataframe
    WSJ = ["Wall Street Journal"] * len(WSJ_article_content)
    data = {"Title" : WSJ_title, "Text" : WSJ_article_content, "Date" : WSJ_date, "News Paper" : WSJ, "Link" : WSJ_link}
    Wall_Street_Journal = pd.DataFrame(data)
    
    
    start_date = start_date.strftime("%d%m%Y")
    end_date = end_date.strftime("%d%m%Y")
    path = f"Data/Wall_Street_Journal_{start_date}_{end_date}.csv"
    Wall_Street_Journal.to_csv(path)

## 2.2. Washington Post

### 2.2.1. Scraping Function

In [2]:
#search_term = term you want to search articles on (" " as input should render all articles)
#start_date = 
#end_Date = 
#username = the username of you account 
#password = password of you account
#-> account will be needed to have access to the articles 


def getWashingtonPost(search_term, start_date, end_date, username, password):
    #put dates in date format
    start_date = datetime.strptime(start_date, "%d/%m/%Y")   
    end_date = datetime.strptime(end_date, "%d/%m/%Y")

    
    #put them in correct format
    
    
    #Declare all list variables for the output
    WP_title = []
    WP_article_content = []
    WP_date = []
    WP_link = []

    #Declare the driver
    #fireFoxOptions = webdriver.FirefoxOptions()
    #fireFoxOptions.add_argument("--headless")
    #driver_WSJ = webdriver.Firefox(options=fireFoxOptions)
    driver_WSJ = webdriver.Firefox()
    driver.get("https://sso.accounts.dowjones.com/login-page?op=localop&scope=openid%20idp_id%20roles%20email%20given_name%20family_name%20djid%20djUsername%20djStatus%20trackid%20tags%20prts%20suuid%20createTimestamp&client_id=5hssEAdMy0mJTICnJNvC9TXEw3Va7jfO&response_type=code&redirect_uri=https%3A%2F%2Faccounts.wsj.com%2Fauth%2Fsso%2Flogin&nonce=ab6a473a-cfa6-4714-8fad-b6dff98f5f18&ui_locales=en-us-x-wsj-223-2&mars=-1&ns=prod%2Faccounts-wsj&state=8rChOTDzC_Y_AK-i.TJAixN_XjsWxwUEEPoHg2OPCaX6qRBu4nGSk5fqLliY4H0B5F7gj_57-XH-YBWGS&protocol=oauth2&client=5hssEAdMy0mJTICnJNvC9TXEw3Va7jfO#!/signin")

    #Go the the searchpage of the Washington Post
    driver.get("https://www.washingtonpost.com/search/?query=+&facets=%7B%22time%22%3A%22all%22%2C%22sort%22%3A%22relevancy%22%2C%22section%22%3A%5B%5D%2C%22author%22%3A%5B%5D%7D")

    #Accept Cookies
    WebDriverWait(driver,10).until(EC.presence_of_element_located((By.ID, "onetrust-accept-btn-handler"))).click()

    #Log-in into Washington Post account
    try: #it is possible that account is already logged in! 

        #Sign in into Washington Post account
        #Click on Sign In
        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH, "//*[@id='__next']/div[1]/nav/div[4]/div[2]/a/p"))).click()

        #located username field
        time.sleep(2) #field needs to completely load
        username_field = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH, "//*[@id='username']")))
        username_field.send_keys(username) #send username
        username_field.send_keys(Keys.RETURN) #press enter

        #Located password field
        time.sleep(2) #field needs to completely load
        password_field = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH, "//*[@id='password']")))
        password_field.send_keys(password)
        password_field.send_keys(Keys.ENTER)

        #Go back to the search page
        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.CLASS_NAME, "wpds-c-jlBemH "))).click()
        go_to_search = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.ID, "query")))
        go_to_search.send_keys(search_term)
        go_to_search.send_keys(Keys.ENTER)

    except:
        #you already logged-in -> continue 
        pass
    
    driver.get("https://www.washingtonpost.com/search/?query=+&facets=%7B%22time%22%3A%22all%22%2C%22sort%22%3A%22relevancy%22%2C%22section%22%3A%5B%5D%2C%22author%22%3A%5B%5D%7D")
    scrape_time = time.time()
    for single_date in daterange(start_date, end_date):
        date = single_date.strftime("%m/%d/%y")

        #Locate the searchbar, send searchterm and press enter
        searchbar = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.CLASS_NAME, "aa-Input")))
        searchbar.send_keys(" ")
        searchbar.send_keys(Keys.ENTER)

        #Select time period
        #select periode specification
        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH, "//*[@id='main-content']/div[1]/section[2]/div[1]/button/div/div[2]/span"))).click()

        #select custom time
        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH, "//*[@id='main-content']/div[1]/section[2]/div[1]/div/div/div[1]/button/div/div[2]"))).click()
        time.sleep(1)

        #Send start date
        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH, "//*[@id='main-content']/div[1]/section[2]/div[1]/div/div/div[2]/div[1]/div/div/div[1]/input"))).send_keys(date)

        #Empty automatic end date
        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH, "//*[@id='main-content']/div[1]/section[2]/div[1]/div/div/div[2]/div[2]/div/div/div[1]/input"))).clear()

        #Send end date
        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH, "//*[@id='main-content']/div[1]/section[2]/div[1]/div/div/div[2]/div[2]/div/div/div[1]/input"))).send_keys(date)

        #Press Apply
        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH, "//*[@id='main-content']/div[1]/section[2]/div[1]/div/div/div[2]/button[2]"))).click()
        time.sleep(5)
        
        #click to load more on the page untill no longer possible
        loading = True
        while(loading):
            try:
                WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH, "//*[@id='main-content']/div[1]/section[3]/button"))).click()
            except:
                loading = False

        articles = WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.TAG_NAME, "article")))

        for a in range(0,len(articles)):
            WP_link.append(articles[a].find_element(By.TAG_NAME, "a").get_attribute('href'))
            WP_date.append(single_date.strftime("%d/%m/%Y"))
    

    #go through all the articles and scrape the content using their url
    #use while loop to prevent one time page error
    for u in range(0, len(WP_link)):
        trys = 0
        while(trys < 100):
            try:
                #go to every article
                driver.get(WP_link[u])
                break
            except:
                trys += 1

        text = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.TAG_NAME, "html"))).text
        try:
            text = text[text.index("Share")+len("Share")+1:text.rindex("Gift Article")]
            WP_article_content.append(text)
        except:
            try:
                text = text[text.index("Share")+len("Share")+1:text.rindex("Comments")]
                WP_article_content.append(text)
            except:
                try:
                    text = text[text.index("tb.boedttibo")+len("tb.boedttibo")+1:text.rindex("Comments")]
                    WP_article_content.append(text)
                except:
                    WP_article_content.append("ERROR")
                    pass
        #idea is: We take the full text and before we have the article content the last pice of text is "share" and immediatly after the content
        #we have "comments" this way we are able to only extract the article content
        #the reasons for this is that each article has a more or less different html dependent on their "type"

        #there are different formats of titles, in this order all titles are located
        try:
            title = driver.find_element(By.CSS_SELECTOR, "#main-content > span:nth-child(2)").text
            WP_title.append(title)
        except:
            try:
                title = driver.find_element(By.CSS_SELECTOR, "#main-content").text
                WP_title.append(title)
            except:
                try:
                    title = driver.find_element(By.CSS_SELECTOR, "#main-content > span").text
                    WP_title.append(title)
                except:
                    WP_title.append("ERROR")
                    pass

    #go back to the search page
    driver.get("https://www.washingtonpost.com/search/?query=+&facets=%7B%22time%22%3A%22all%22%2C%22sort%22%3A%22relevancy%22%2C%22section%22%3A%5B%5D%2C%22author%22%3A%5B%5D%7D")

    #Close driver
    driver.quit()

    #How long did the scraping took? 
    print("--- %s seconds ---" % (time.time() - scrape_time))

    #Create output file
    #Make the dataframe
    WP = ["Washington Post"] * len(WP_article_content)
    data = {"Title" : WP_title, "Text" : WP_article_content, "Date" : WP_date, "News Paper" : WP, "Link" : WP_link}
    WashingtonPost = pd.DataFrame(data)
    
    start_date = start_date.strftime("%d%m%Y")
    end_date = end_date.strftime("%d%m%Y")
    path = f"Washington_Post_{start_date}_{end_date}.csv"
    WashingtonPost.to_csv(path)

## 2.3. New York Times

### 2.3.1. Scraping Function

In [29]:
def getNewYorkTimes(start_date, end_date, username, password):
    #put dates in date format
    start_date = datetime.strptime(start_date, "%d/%m/%Y") 
    end_date = datetime.strptime(end_date, "%d/%m/%Y")

    #Declare all list variables for the output
    NYT_title = []
    NYT_article_content = []
    NYT_date = []
    NYT_link = []

    #set up driver
    #driver = webdriver.Chrome()
    driver = webdriver.Firefox()
    #go to log-in screen of NYT
    driver.get("https://myaccount.nytimes.com/auth/login?response_type=cookie&client_id=vi&redirect_uri=https%3A%2F%2Fwww.nytimes.com%2Fsubscription%2Fonboarding-offer%3FcampaignId%3D7JFJX%26EXIT_URI%3Dhttps%253A%252F%252Fwww.nytimes.com%252F&asset=masthead")
    driver.get("https://myaccount.nytimes.com/auth/login?response_type=cookie&client_id=vi&redirect_uri=https%3A%2F%2Fwww.nytimes.com%2Fsubscription%2Fonboarding-offer%3FcampaignId%3D7JFJX%26EXIT_URI%3Dhttps%253A%252F%252Fwww.nytimes.com%252F&asset=masthead")
    driver.get("https://myaccount.nytimes.com/auth/login?response_type=cookie&client_id=vi&redirect_uri=https%3A%2F%2Fwww.nytimes.com%2Fsubscription%2Fonboarding-offer%3FcampaignId%3D7JFJX%26EXIT_URI%3Dhttps%253A%252F%252Fwww.nytimes.com%252F&asset=masthead")

    #not a robot
    input("Press Enter to continue...")

    #Select log-in field
    login = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.ID, "email")))

    #Send username
    login.send_keys(username)

    #Press continue
    WebDriverWait(driver,10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#myAccountAuth > div > div > div > form > div > div.css-bho3kg-buttonWrapper-buttonStyles-Button > button"))).click()

    #Select password field
    password_field = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.ID, "password")))

    #send password
    password_field.send_keys(password)

    #Press continue
    WebDriverWait(driver,10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#myAccountAuth > div > div > form > div > div.css-1nkv26b-buttonWrapper-buttonStyles-Button > button"))).click()

    time.sleep(1)

    #go to search page
    driver.get("https://www.nytimes.com/search?dropmab=false&query=&sort=best")

    #Accept Cookies
    try:
        WebDriverWait(driver,5).until(EC.presence_of_element_located((By.CLASS_NAME, "banner__container__cta--accept"))).click()
    except:
        pass

    scrape_time = time.time()
    for single_date in daterange(start_date, end_date):
        date = single_date.strftime("%Y%m%d")

        url = f"https://www.nytimes.com/search?dropmab=false&endDate={date}&query=%20&sort=best&startDate={date}"
        driver.get(url)

        #click to load more on the page untill no longer possible
        loading = True
        while(loading):
            try:
                #time.sleep(random.randint(2,4))
                WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH, "//*[@id='site-content']/div/div[2]/div[2]/div/button"))).click()
            except:
                loading = False

        articles = WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "css-1l4w6pd")))
        dates = WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "css-17ubb9w")))

        date_list = WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "css-17ubb9w")))
        true_date = date_list[0].text
        for a in range(0, len(articles)):
            if(true_date == date_list[a].text):
                NYT_link.append(articles[a].find_element(By.TAG_NAME, "a").get_attribute('href'))
                NYT_date.append(single_date.strftime("%d/%m/%Y"))
    
    #go through all the articles urls
    for u in range(0, len(NYT_link)):
        driver.get(NYT_link[u])
        try:
            WebDriverWait(driver,3).until(EC.presence_of_element_located((By.CLASS_NAME, "banner__container__cta--accept"))).click()
        except:
            pass

        try:
            WebDriverWait(driver,1).until(EC.presence_of_element_located((By.CLASS_NAME, "loginButton"))).click()
            driver.switch_to.window( driver.window_handles[1])

            #Select log-in field
            login = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.ID, "email")))
            login.send_keys(username)

            #Press continue
            WebDriverWait(driver,10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#myAccountAuth > div > div > div > form > div > div.css-bho3kg-buttonWrapper-buttonStyles-Button > button"))).click()

            #Select password field
            password_field = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.ID, "password")))

            #send password
            password_field.send_keys(password)

            #Press continue
            WebDriverWait(driver,10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#myAccountAuth > div > div > form > div > div.css-1nkv26b-buttonWrapper-buttonStyles-Button > button"))).click()            

            time.sleep(3)
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
            driver.get(article_urls[u])

        except:
            pass
        #get the article title
        try:
            title = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.TAG_NAME, "h1"))).text
            NYT_title.append(title)
        except:
            NYT_title.append("ERROR")

        #get article content
        try:
            content = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.TAG_NAME, "article")))
            content = content.find_elements(By.TAG_NAME, "p")
            text = ""
            for t in range(0,len(content)):
                text += content[t].text

            NYT_article_content.append(text)
        except:
            NYT_article_content.append("ERROR")

    #How long did the scraping took? 
    print("--- %s seconds ---" % (time.time() - scrape_time))

    driver.quit()

    NYT = ["New York Times"] * len(NYT_article_content)
    data = {"Title" : NYT_title, "Text" : NYT_article_content, "Date" : NYT_date, "News Paper" : NYT, "Link" : NYT_link}
    NewYorkTimes = pd.DataFrame(data)
    
    start_date = start_date.strftime("%d%m%Y")
    end_date = end_date.strftime("%d%m%Y")
    path = f"Data/New_York_Times_{start_date}_{end_date}.csv"
    NewYorkTimes.to_parquet(path)

## 2.4. Politico

### 2.4.1. Scraping Function

In [15]:
#start_date and end_date should be in format m/d/y as a string

def getPolitico(start_date, end_date):
    #put dates in date format
    end_date = datetime.strptime(end_date, "%d/%m/%Y")
    start_date = datetime.strptime(start_date, "%d/%m/%Y")
    
    #Declare all list variables for the output
    P_title = []
    P_article_content = []
    P_date = []
    P_link = []

    driver = webdriver.Firefox()

    #go to the website
    driver.get("https://www.politico.com/search?adv=true")

    #accept cookies
    try:
        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.ID, "onetrust-accept-btn-handler"))).click()
    except:
        pass

    scrape_time = time.time()
    for single_date in daterange(start_date, end_date):
        date = single_date.strftime("%m/%d/%Y")
        driver.get(f"https://www.politico.com/search/1?adv=true&start={date}&end={date}")
        articles = WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.TAG_NAME, "article")))
        for a in range(0, len(articles)):
            P_link.append(articles[a].find_element(By.TAG_NAME, "a").get_attribute('href'))
            P_date.append(date)

        #get the number of pages to know how much you should click 
        try:
            nr_pages = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".pagination > ol:nth-child(2) > li:nth-child(6) > a:nth-child(1)"))).text
        except:
            nr_pages = 1
            pass

        for p in range(1, int(nr_pages)):
            url = f"https://www.politico.com/search/{p+1}?adv=true&start={date}&end={date}"
            driver.get(url)

        articles = WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.TAG_NAME, "article")))
        for a in range(0, len(articles)):
            P_link.append(articles[a].find_element(By.TAG_NAME, "a").get_attribute('href'))
            P_date.append(single_date.strftime("%d/%m/%Y"))

    #loop through every article
    for u in range(0,len(P_link)):
        driver.get(P_link[u])

        #get the title
        try:
            title = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.CLASS_NAME, "headline"))).text
            P_title.append(title)
        except:
            P_title.append("PRO")
            pass

        #get content
        try:
            content = WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "story-text__paragraph  ")))
            text = ""
            for c in range(0,len(content)):
                text += content[c].text

            P_article_content.append(text)

        except:
            P_article_content.append("PRO")
            pass

    driver.quit()
    print("--- %s seconds ---" % (time.time() - scrape_time))

    P = ["Politico"] * len(P_article_content)
    data = {"Title" : P_title, "Text" : P_article_content, "Date" : P_date, "News Paper" : P, "Link" : P_link}
    Politico = pd.DataFrame(data)

    start_date = start_date.strftime("%d%m%Y")
    end_date = end_date.strftime("%d%m%Y")
    path = f"Politico_{start_date}_{end_date}.csv"
    Politico.to_csv(path)
    

# 3. Running Functions

getWallStreetJournal(username, password, start_date, end_date)

getWashingtonPost(search_term, start_date, end_date, username, password)

getNewYorkTimes(start_date, end_date, username, password):

getPolitico(start_date, end_date):

In [31]:
getWallStreetJournal("reane.delaunoy@telenet.be", "REenJUC0MB0", "01/01/2010", "01/01/2010")

01/01/2010
--- 394.8229627609253 seconds ---


In [42]:
getWashingtonPost(" ", "01/01/2022", "01/01/2022", "tb.boedttibo@gmail.com", "ThesisR&T")

--- 168.77081990242004 seconds ---


In [30]:
getNewYorkTimes("01/01/2022", "31/01/2022", "tb.boedttibo@gmail.com", "ThesisR&T0")

Press Enter to continue...
--- 33978.41111564636 seconds ---


In [ ]:
getPolitico("01/01/2022", "01/01/2022")

In [32]:
test = pd.read_parquet("Data/New_York_Times_01012022_31012022.csv")